In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings

folder_path = "/content/drive/MyDrive/model_imp_whole/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 2.32 s, sys: 426 ms, total: 2.75 s
Wall time: 5.94 s


In [3]:
%%time
becodemixed_data =  pd.read_excel('/content/drive/MyDrive/model_imp_whole/becodemixed_data_2.xlsx') 

CPU times: user 540 ms, sys: 35.5 ms, total: 576 ms
Wall time: 1.28 s


In [4]:
becodemixed_data['enc_label'] = becodemixed_data['Label'].replace({'Neutral' :6 ,'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})

In [5]:
print("Size of the training data ==> ", becodemixed_data.shape)

Size of the training data ==>  (2501, 3)


In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [7]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(becodemixed_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

train_data= pd.concat([becodemixed_data, encoded_labels ], axis=1)

In [8]:
becodemixed_data = becodemixed_data.rename(columns={'Label_Neutral' :'Neutral' ,'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
becodemixed_data.head(50)

,Text,Label,enc_label
0,K jabar bole j bacha chhara sob pare ... tar o...,Disgust,3
1,"Definitely devastated film ta dekhar por , jus...",Anger,4
2,Boddo hingshute hoye gechs tui or sathe mishe ...,Disgust,3
3,@ ipshita _ ahsan Apu tumi etodin kothay Chile...,Sadness,1
4,Meyeta ke help korlam kntu ekbaro takeo dklo n...,Disgust,3
5,"Irrfan Khan hollywood e abar dekha debe , trai...",Joy,0
6,Dorshok ra was literally furious after watchin...,Surprise,2
7,Ramdhanu ended with tears but approach ta besh...,Sadness,1
8,Mejo Jethu ar Boro Jethu er majhe amay boshiye...,Neutral,6
9,"Editing was horrible , thik thak editing thakl...",Disgust,3


In [32]:
import re

count = 0 ;

for text in becodemixed_data['Text']:
  becodemixed_data['Text'][count] = re.sub('@ [^\s]+','',str(text));
  count = count + 1;

  
becodemixed_data.head(5)

,Text,Label,enc_label,Texts,cleaned
0,K jabar bole j bacha chhara sob pare ... tar o...,Disgust,3,K jabar bole j bacha chhara ... opr Latin form...,k jabar bole j bacha chhara opr latin form can...
1,"Definitely devastated film ta dekhar por , jus...",Anger,4,"Definitely devastated film dekhar , just kore ...",definitely devastated film dekhar just kore ba...
2,Boddo hingshute hoye gechs tui or sathe mishe ...,Disgust,3,Boddo hingshute gechs tui or sathe mishe mishe...,boddo hingshute gechs tui or sathe mishe mishe...
3,_ ahsan Apu tumi etodin kothay Chiley ?? Toma...,Sadness,1,_ ahsan Apu etodin kothay Chiley ?? Tomake mis...,ahsan apu etodin kothay chiley tomake miss kor...
4,Meyeta ke help korlam kntu ekbaro takeo dklo n...,Disgust,3,Meyeta ke help korlam kntu ekbaro takeo dklo e...,meyeta ke help korlam kntu ekbaro takeo dklo e...


In [ ]:
 becodemixed_data.to_excel('becodemixed_data_clean_2.xlsx')
!cp becodemixed_data_clean_2.xlsx "drive/My Drive/"

In [11]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [28]:
def get_becodemixed_stop_words():
    #swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("/content/drive/MyDrive/model_imp/custom_becodemixed_stopwords.txt",  header=None) 
    #sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    #tamil_stop_words = list(set(sw1 + sw2))
    becodemixed_stop_words = list(set(sw2))
    return becodemixed_stop_words

In [29]:
# #removing URLs and Punctuations
# becodemixed_data['cleaned'] = becodemixed_data['Texts'].apply(remove_url)
# #Removing punctuations


becodemixed_stop_words = get_becodemixed_stop_words()
becodemixed_data['Texts'] = becodemixed_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
becodemixed_data['cleaned'] = becodemixed_data['Texts'].apply(text_cleaning)


# valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
# train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
# test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))

# train_data['cleaned'] = train_data['Texts'].apply(text_cleaning)
# valid_data['cleaned'] = valid_data['Texts'].apply(text_cleaning)
# test_data['cleaned'] = test_data['Texts'].apply(text_cleaning)


## Data samples after cleaning
print("Original Data:===\n",becodemixed_data.Text[0],"\nCleaned Data:===\n",becodemixed_data.cleaned[0],)

Original Data:===
 K jabar bole j bacha chhara sob pare ... tar opr abar Latin form ?? Can u pls explain  https :// t . co / 8xSl903uam 
Cleaned Data:===
 k jabar bole j bacha chhara opr latin form can u pls explain https t co 8xsl903uam


In [ ]:
valid_data['cleaned'].shape